In [21]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [2]:
dat = pd.read_csv("iris.csv")
dat.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [14]:
dat = dat[dat["Species"] != "setosa"]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor
52,6.9,3.1,4.9,1.5,versicolor
53,5.5,2.3,4.0,1.3,versicolor
54,6.5,2.8,4.6,1.5,versicolor
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [23]:
X = dat[['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width']]
y = dat['Species']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test) 

Accuracy: 1.0


In [33]:
pd.DataFrame({'test': y_test, 'predict': y_pred})

,test,predict
73,versicolor,versicolor
18,setosa,setosa
118,virginica,virginica
78,versicolor,versicolor
76,versicolor,versicolor
31,setosa,setosa
64,versicolor,versicolor
141,virginica,virginica
68,versicolor,versicolor
82,versicolor,versicolor


In [34]:
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Accuracy: 1.0


## ex3. Search the best K

In [35]:
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(estimator = knn,
    param_grid = {'n_neighbors' : list(range(1,10))},
    scoring = 'accuracy',
    cv = 10,
    refit = True)
best_model = gs.fit(X_train, y_train).best_estimator_
print('Best k : %d' % best_model.get_params()['n_neighbors'])

Best k : 9
